In [3]:
!mkdir /root/.ssh
!mv /content/id_rsa /root/.ssh/id_rsa
!chmod 600 /root/.ssh/id_rsa
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!git clone git@github.com:TimothyAlexisVass/udi.git
!rm -r sample_data

mkdir: cannot create directory ‘/root/.ssh’: File exists
# github.com:22 SSH-2.0-babeld-dc5ec9be
# github.com:22 SSH-2.0-babeld-6706de95
# github.com:22 SSH-2.0-babeld-f8b1fc6c
# github.com:22 SSH-2.0-babeld-f8b1fc6c
# github.com:22 SSH-2.0-babeld-f8b1fc6c
Cloning into 'udi'...
remote: Enumerating objects: 901, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 901 (delta 33), reused 64 (delta 24), pack-reused 820
Receiving objects: 100% (901/901), 173.10 KiB | 344.00 KiB/s, done.
Resolving deltas: 100% (473/473), done.
rm: cannot remove 'sample_data': No such file or directory


In [2]:
%cd /content/udi
!git pull

[Errno 2] No such file or directory: '/content/udi'
/content
fatal: not a git repository (or any of the parent directories): .git


In [5]:
!lsof -ti:5000 | xargs kill
!python3 /content/udi/launch --share


Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -q, --queue <value>    integer value to be sent with the signal
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
Installing git+https://github.com/TimothyAlexisVass/diffusers...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
18:21:17 system   | web.1 started (pid=4038)
18:21:17 system   | worker.1 started (pid=4036)
18:21:17 system   | share.1 started (pid=4042)
18:21:18 web.1    |  * Serving Flask app 'app'
18:21:18 web.1    |  * Debug mode: on
18:21:18 web.1    | WARNING: This is a development serv

KeyboardInterrupt: ignored

In [9]:
import os
import sys
import json
import time
import torch
import random
from udi.worker.prompt_manager import process_prompt
from udi.worker.model_manager import ModelManager
from udi.worker.filters import latents_filter
from PIL.PngImagePlugin import PngInfo

model_manager = ModelManager()
numpy_to_pil = model_manager.txt2img.numpy_to_pil

script_directory = '/content/udi/worker'
main_app_directory = os.path.dirname(script_directory)

creations_directory = os.path.join(main_app_directory, 'static/creations')
preview_directory = os.path.join(main_app_directory, 'static/preview')
active_directory = os.path.join(main_app_directory, 'worker/queue/active')
done_directory = os.path.join(main_app_directory, 'worker/queue/done')
new_directory = os.path.join(main_app_directory, 'worker/queue/new')
os.makedirs(creations_directory, exist_ok=True)
os.makedirs(preview_directory, exist_ok=True)
os.makedirs(active_directory, exist_ok=True)
os.makedirs(done_directory, exist_ok=True)
os.makedirs(new_directory, exist_ok=True)
processing_file = os.path.join(preview_directory, 'processing.json')

def console_log(message):
    print(str(message) + "\n")
def console_error(error):
    print(str(error) + "\n")
def log_process(data):
    with open(processing_file, 'w') as json_file:
        json.dump(data, json_file)

@torch.no_grad()
def decode_latents(latents):
    scaling = 4.444 + saturation / 16

    samples = model_manager.vae.decode(latents * scaling).sample
    if normalize_latents:
        samples = normalize_tensor(samples)
    else:
        samples = samples.mul(contrast/100).add(brightness/100).clamp(0, 1)
    return numpy_to_pil(samples.permute(0, 2, 3, 1).cpu().numpy())

# Globals
num_inference_steps = 20

def inference_callback(index, timestep, latents):
    step = index + 1
    latents = latents_filter(latents, timestep, filter_type, kernel_size=filter_kernel_size, sigma=filter_spread, filter_strength=filter_strength)
    preview = decode_latents(latents)[0]

    if step < num_inference_steps:
        preview.save(os.path.join(preview_directory, f"{step}.jpg"), quality=70)
        log_process({'step': step, 'timestep': timestep.item(), 'num_inference_steps': num_inference_steps})
    previous_file = os.path.join(preview_directory, f"{index}.jpg")
    if os.path.exists(previous_file):
        os.remove(previous_file)

    console_log(f"Step: {step} / {num_inference_steps} ({timestep.item()})")

    return {"latents": latents}

pipeline_defaults = {
    'num_images_per_prompt': 1,
    'output_type': 'latent',
    'callback': inference_callback,
    'callback_steps': 1,
    'guidance_rescale': 0.7,
}

width_and_height = [
    [1536, 640],
    [1344, 768],
    [1280, 832],
    [1152, 896],
    [1024, 1024],
    [896, 1152],
    [832, 1280],
    [768, 1344],
    [640, 1536],
]


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
job_task = {'pipeline': {'prompt': 'A beautiful woman', 'negative_prompt': 'blurry, deformed, disfigured, bad anatomy, ugly', 'resolution': 4, 'guidance_scale': 8.5, 'num_inference_steps': 20, 'seed': 0}, 'inference': {'normalize_latents': False, 'upscale_result': True, 'use_refiner': True, 'saturation': 50, 'brightness': 50, 'contrast': 50, 'filter_type': 'sharpen', 'filter_strength': 5, 'filter_spread': 3, 'filter_kernel_size': 13}, 'scene': {'lighting': 'Low-Key Lighting', 'environment': '', 'ambiance': '', 'composition': '', 'elements': '', 'mood': 'Serene', 'emotion': 'Joy'}, 'view': {'shot_framing': 'Leading Lines Shot', 'focus': '', 'shot_size': 'Close-Up', 'view_angle': '', 'perspective': ''}, 'style': {'artist': '', 'medium': 'Photograph', 'technique': '', 'color_palette': 'Vibrant', 'art_movement': '', 'genre': 'Sci-fi', 'time_period': ''}, 'quality': {'quality_prompt': '', 'word_salad': {'superb': 1, 'photorealistic': 1, 'masterpiece': 1, 'intricate details even to the smallest particle': 1, 'extreme detail of the environment': 1, 'beautifully rendered textures and colors': 1, 'crisp edges': 1, 'high quality': 1}}, 'camera': {'model': 'Nikon D850', 'shutter_speed': '', 'focal_length': 'Ultra-Wide-Angle 20mm', 'aperture': '', 'sensor_format': '', 'sensitivity': '', 'film_format': '', 'film_stock': 'Kodak Portra 400'}}

global num_inference_steps

###### Define variables ###############################
pipeline = job_task["pipeline"]
scene = job_task["scene"]
view = job_task["view"]
camera = job_task["camera"]
quality = job_task["quality"]
style = job_task["style"]

num_inference_steps = pipeline["num_inference_steps"]

###### Set seed and resolution parameters #############
seed = pipeline.get("seed", 0)
seed = random.randint(1, 2147483647) if seed == 0 else seed
pipeline["generator"] = torch.manual_seed(seed)
pipeline["width"], pipeline["height"] = width_and_height[pipeline.get("resolution", 4)]
if "seed" in pipeline:
    del pipeline["seed"]
if "resolution" in pipeline:
    del pipeline["resolution"]

console_log("Setting global inference parameters")
inference = {}
for inference_setting, value in job_task["inference"].items():
    globals()[inference_setting] = inference[inference_setting] = value

console_log("Building prompt...")
###### Build prompt ###################################
console_log("...scene")
if scene:
    pipeline["prompt"] += " " + " ".join(scene.values())

console_log("...view")
if view:
    view_prompt = [view.get("shot_size", ""), view.get("focus", ""), view.get("shot_framing", "")]
    if view["view_angle"]:
        view_prompt.append(f"Taken from a {view['view_angle']}")
    if view["perspective"]:
        view_prompt.append(f"with {view['perspective']}")
    pipeline["prompt"] = "(" + " ".join(view_prompt) + ")1.0 " + pipeline["prompt"]

console_log("...camera")
if camera:
    camera_prompt = []
    if camera["model"]:
        camera_prompt.append(f"taken with {camera['model']}")
    camera_prompt.append(camera.get("shutter_speed", ""))
    camera_prompt.append(camera.get("aperture", ""))
    camera_prompt.append(camera.get("focal_length", ""))
    camera_prompt.append(camera.get("sensor_format", ""))
    camera_prompt.append(camera.get("sensitivity", ""))
    if camera["film_format"]:
        camera_prompt.append(f"({camera['film_format']} film)1.0")
    camera_prompt.append(camera.get("film_format", ""))
    if camera["focal_length"]:
        camera_prompt.append(f"({camera['focal_length']} lens)1.0")
    pipeline["prompt"] = "(" + " ".join(camera_prompt) + ")1.0 " + pipeline["prompt"]

console_log("...quality")
if quality:
    if quality["word_salad"]:
        quality_string = " ".join(f"({setting}){'{0:.1f}'.format(weight)}" for setting, weight in quality["word_salad"].items())
    else:
        quality_string = ""
    pipeline["prompt"] += f" {quality['quality_prompt']} " + quality_string

console_log("...style")
if style:
    if style["genre"]:
        pipeline["prompt"] = style["genre"] + " " + pipeline["prompt"]
    if style["medium"]:
        pipeline["prompt"] = style["medium"] + " " + pipeline["prompt"]
    if style["color_palette"]:
        pipeline["prompt"] += f" {style['color_palette']} colors"
    style_prompt = [style.get("artist", ""), style.get("technique", ""), style.get("art_movement", ""), style.get("time_period", "")]
    pipeline["prompt"] += " " + " ".join(style_prompt)

###### Run Inference ##################################
prompt_1 = pipeline.pop("prompt").replace("  ", " ")
negative_prompt_1 = pipeline.pop("negative_prompt")


console_log("Running text to image model")
_ = model_manager.txt2img.to("cuda")
prompt_embeds = process_prompt(model_manager.txt2img, prompt_1=prompt_1, negative_prompt_1=negative_prompt_1)
prompt_embed_parameters = {
    "prompt_embeds": prompt_embeds["positive"]["embeds"],
    "pooled_prompt_embeds": prompt_embeds["positive"]["pooled"],
    "negative_prompt_embeds": prompt_embeds["negative"]["embeds"],
    "negative_pooled_prompt_embeds": prompt_embeds["negative"]["pooled"]
}
console_log("Setting parameters")
parameters = {**prompt_embed_parameters, **pipeline, **pipeline_defaults}
console_log("Generating latents")
latents = model_manager.txt2img(**parameters, denoising_end=0.8 if use_refiner else 1.0).images

if use_refiner:
    console_log("Switching to refiner")
    _ = model_manager.refiner.to("cuda")
    prompt_embeds = process_prompt(model_manager.refiner, prompt_1=prompt_1, negative_prompt_1=negative_prompt_1)
    prompt_embed_parameters = {
        "prompt_embeds": prompt_embeds["positive"]["embeds"],
        "pooled_prompt_embeds": prompt_embeds["positive"]["pooled"],
        "negative_prompt_embeds": prompt_embeds["negative"]["embeds"],
        "negative_pooled_prompt_embeds": prompt_embeds["negative"]["pooled"]
    }
    console_log("Setting parameters")
    parameters = {**prompt_embed_parameters, **pipeline, **pipeline_defaults}
    del parameters["width"]
    del parameters["height"]
    console_log("Generating latents")
    latents = model_manager.refiner(**parameters, denoising_start=0.8, image=latents).images

console_log("Decoding latents")
###### Decode and save image with metadata ############
image = decode_latents(latents)[0]

metadata = PngInfo()
metadata.add_text("pipeline", json.dumps(pipeline))
metadata.add_text("scene", json.dumps(scene))
metadata.add_text("view", json.dumps(view))
metadata.add_text("camera", json.dumps(camera))
metadata.add_text("quality", json.dumps(quality))
metadata.add_text("style", json.dumps(style))

console_log("Saving image")
file_name = f"{parameters['prompt'][:100]}_{seed}.png"
image_path = os.path.join(creations_directory, file_name)
image.save(image_path, pnginfo=metadata)
image.save(os.path.join(preview_directory, f"{num_inference_steps}.png"))

torch.cuda.empty_cache()

console_log("Inference completed")

Setting global inference parameters

Building prompt...

...scene

...view

...camera

...quality

...style

Running text to image model



OutOfMemoryError: ignored